## API for Battery Control Agent

### Discretization code only inside function approximation or Q-update module.

In [60]:
import numpy as np
from collections import *
import matplotlib.pyplot as plt
import pandas as pd
import random
import seaborn as sns
sns.set(color_codes=True)
from __future__ import division
#%matplotlib inline


In [64]:
class QLearning:
    def __init__ (self, E_cap, P_cap, efficiency, gamma, learning_rate):
        
        #-----------Battery parameters-----------------
        self.E_max = E_cap
        self.P_cap = P_cap
        self.E_min = (1-0.80)*self.E_max #depth of discharge
        self.E_init = (0.30)*self.E_max #30% of charge every morning 
        self.eta = efficiency
        
        #-----------Q-learning Hyperparameters---------
        self.alpha = learning_rate
        self.gamma = gamma
        self.qvalue = defaultdict(lambda: -0.00 * np.random.randn(len(self.actions)) 0) #negative qvalues for 12 actions
        self.epsilon = 0.1
        #-----------Steps-------------------------------
        self.action_step = 0.10*P_cap
        self.energy_step = 0.30
        self.netload_step = 0.30
        self.actions = np.arange(-self.P_cap, self.P_cap, self.action_step)
                
        #------------Discrete Bins---------------------
        self.energy_bins = np.arange(1.2, self.E_max + 0.01, self.energy_step)
        self.netload_bins = np.arange(-2.6, 5.5, self.netload_step)
        self.price = self.price = np.array([.040,.040,.040,.040,.040,.040,.080,.080,.080,.080,.040,.040,.080,.080,.080,.040,.040,.120,.120,.040,.040,.040,.040,.040]).tolist()

    def get_action(self, currentState):
        '''
        Returns an action according to an epsilon-greedy policy
        Output: action, action_index
        '''
        legal_actions = self.get_legal_actions(currentState)
        currentState_index = self.discretize_state(currentState)
        
        if np.random.random() <= self.epsilon:
            action_index = random.choice(legal_actions)
            action = self.actions[action_index]
        else:
            qlist = self.qvalue[currentState_index]
            qlist = [qlist[k] for k in legal_actions]
            index = np.nanargmax(qlist) 
            action_index = legal_actions[index]
            action = self.actions[action_index]
            
        return action, action_index
    
    def get_legal_actions(self, currentState):
        '''
        Calculate and return allowable action set
        Output: List of indices of allowable actions
        '''
        energy_level = currentState[1]
        #energy_level = self.energy_bins((np.digitize(energy_level, self.energy_bins)-1))
        lower_bound = max(self.E_min - energy_level, -self.P_cap)
        upper_bound = min(self.E_max - energy_level, self.P_cap)
        max_bin = int(np.digitize(upper_bound, self.actions, right=True)) ###
        min_bin = int(np.digitize(lower_bound, self.actions, right=True)) ###
        
        legal_actions = []
        
        for k in range(min_bin, max_bin+1):
            legal_actions.append(k)
        return legal_actions
     
    def discretize_state(self, state):
        '''
        Discretize state variables individually and return an index tuple for hashing Q[(s,a)] function.
        Return: bin values.
        '''
        load_index = int(np.digitize(state[0], self.netload_bins, right=True)) ###
        energy_index = int(np.digitize(state[1], self.energy_bins, right=True)) ###
        state_index = (load_index, energy_index)
        
        return state_index
    
    def next_step(self, currentState, action, time_step):
        '''
        Perform constraint checking (energy, grid) and assign penalty/rewards.
        Output: reward/penalty, next state, constraint satisfaction (boolean)
        '''
        current_netload = currentState[0]
        current_energy = currentState[1]
        
        if action >= 0:
            P_charge, P_discharge = action, 0.0
        else:
            P_charge, P_discharge = 0.0, action
        
        E_next = current_energy + self.eta * P_charge + P_discharge
        P_grid = current_netload + P_charge + P_discharge
        
        condition1 = bool((E_next >= self.E_min) and (E_next <= self.E_max))

        if condition1 != True:
            print action, E_next
            print 'hello'
        condition2 = bool(P_grid >= 0)
        #condition = (P_grid >= 0)
        
        if P_grid >= 0:
            reward = -P_grid*self.price[time_step] # -P_grid*self.price[time_step]
            #print current_energy, action, E_next
            nextState = (0.0, E_next)
        elif P_grid < 0:
            reward = P_grid*10
            nextState = None
        
        condition = condition1 and condition2
        
        return reward, nextState, P_grid, condition
    
    def update_Qvalue(self, currentState, action_index, reward, nextState, condition):
        '''
        Performs a Q-learning update.
            if condition = False, penalize
            if condition = True, update Q-value. 
        
        :type currentState, nextState: list of ints
        :param currentState, nextState: 

        :type reward: float
        :param reward: immediate reward associated with transitioning to nextState
        
        '''
        if condition == False:
            currentState_index = self.discretize_state(currentState)
            td_error = reward - self.qvalue[currentState_index][action_index]
            self.qvalue[currentState_index][action_index] = 0.50*td_error
            return None
        
        currentState_index = self.discretize_state(currentState)
        nextState_index = self.discretize_state(nextState)
        nextAction_index = np.nanargmax(self.qvalue[nextState_index])
        
        td_error = self.gamma*self.qvalue[nextState_index][nextAction_index] - self.qvalue[currentState_index][action_index] 
        self.qvalue[currentState_index][action_index] += self.alpha *(reward + td_error) 
        
        return None
    
    def update_Qvalue_approx(self, currentState, action, reward, nextState, condition):
        '''
        Use function approximation to get Q-value of currentState and perform Q-update.
        Q table = defaultdict.
        '''
        return None
    
    def greedy_action(self, currentState):
        currentState_index = self.discretize_state(currentState)
        legal_actions = self.get_legal_actions(currentState)
        qlist = self.qvalue[currentState_index]
        qlist = [qlist[k] for k in legal_actions]
        index = np.nanargmax(qlist) 
        action_index = legal_actions[index]
        action = self.actions[action_index]
        return action, action_index


SyntaxError: invalid syntax (<ipython-input-64-7e5d915023b3>, line 14)

In [65]:
if __name__ == "__main__":
    
    def episode(day_states, ql, grid_profile):
        currentState = day_states[0]
        currentState[1] = ql.E_init
        grid_list = grid_profile
        reward_list = list() #episodic rewards
        cumulative_reward = 0.0
        
        for t in range(23):
            time_step = t+1
            action, action_index = ql.get_action(currentState)
            reward, nextState, P_grid, condition = ql.next_step(currentState, action, t)
            
            if nextState != None:
                E_next = nextState[1]
                nextState = day_states[time_step]
                nextState[1] = E_next            
            
            ql.update_Qvalue(currentState, action_index, reward, nextState, condition)
            
            if condition == False:
                grid_list.append(P_grid)
                break
            
            currentState = nextState
            grid_list.append(P_grid)
            cumulative_reward += reward

        reward_list.append(cumulative_reward)
        return grid_list, cumulative_reward
    
    ql = QLearning(6.0, 3.0, 0.90, 0.90, 0.5)
    YEARS = 500
    total_days = 15*YEARS
    df_solar = pd.read_csv('/Users/siddharth481/Documents/IITBombay/Battery/SolarData/solar_clean.csv')
    df_load = pd.read_csv('/Users/siddharth481/Documents/IITBombay/Battery/SolarData/load_data_peak6.csv')
    diff = (df_load.ix[0:14] - df_solar.ix[0:14])
    net_load = pd.concat(([diff.multiply(1)])*YEARS, ignore_index=True).values.tolist()
    ql.epsilon = 0.7
    count = 1
    grid_profile = list()
    reward_list = list()
    
    for episode_day in xrange(total_days):
        E_st = np.zeros(24)
        weekly_bill = 0.0
        ##S = [net_load[episode_day], E_st, ql.price]
        S = [net_load[episode_day], E_st]
        day_states = map(list, zip(*S))
        grid_profile, reward = episode(day_states, ql, grid_profile)
        
        if count == 7:
            reward_list.append(-1*reward)
            weekly_bill = 0.0
            count = 1
        else:
            count +=1
        
        if ql.epsilon >= 0:
            ql.epsilon -= 1/total_days
    plt.plot(grid_profile)#make the plot
    print ql.epsilon
    plt.show()
    
            

-0.000133333333295


## Policy Testing

In [66]:
total_days = 15
ql = ql
grid_prof = list()
reward_list = list()
action_list = list()

def play_episode(day_states, ql, grid_profile):

    currentState = day_states[0]
    currentState[1] = ql.E_init
    grid_list = grid_profile
    reward_list = list() 
    cumulative_reward = 0.0
    
    for t in range(24):
        time_step = t+1
        action, action_index = ql.greedy_action(currentState)
        index = ql.discretize_state(currentState)
        #print ql.qvalue[index], action_index
        action_list.append(action)
        reward, nextState, P_grid, condition = ql.next_step(currentState, action, t)
        #print currentState[1], action, nextState[1]
        #print '\n'

        if (nextState != None and (time_step < 23)):
            E_next = nextState[1]
            nextState = day_states[time_step]
            nextState[1] = E_next 
            currentState = nextState
            
        if condition == False:
            grid_list.append(P_grid)
            cumulative_reward += reward

        grid_list.append(P_grid)
        cumulative_reward += reward

    reward_list.append(cumulative_reward)
    return grid_list, cumulative_reward

for episode_day in xrange(total_days):
    E_st = np.zeros(24)
    S = [net_load[episode_day], E_st]
    day_states = map(list, zip(*S))
    grid_prof, reward = play_episode(day_states, ql, grid_prof)
        
    '''
    if count == 7:
        rewards.append(weekly_bill)
        weekly_bill = 0.0
        count = 1
    else:
        count +=1
    '''
    reward_list.append(reward)
    

In [67]:
plt.plot(grid_prof)
plt.show()

## Plotting Routine

In [49]:
TOU = [.040,.040,.040,.040,.040,.040,.080,.080,.080,.080,.040,.040,.080,.080,.080,.040,.040,.120,.120,.040,.040,.040,.040,.040]
scaled_TOU = list()
for price in TOU:
    scaled_TOU.append(price*150 - 6)
print scaled_TOU

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.0, 6.0, 6.0, 6.0, 0.0, 0.0, 6.0, 6.0, 6.0, 0.0, 0.0, 12.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [50]:
lower_margin = 3
upper_margin = 7
week = 0 #35*7
plt.plot([x*-1 for x in action_list[24*(week + lower_margin):24*(week + upper_margin)]], 'g')
plt.plot(scaled_TOU*(upper_margin - lower_margin), 'c')
plt.plot(df_solar.iloc[(week + lower_margin):(week + upper_margin)].values.flatten(), 'y')
plt.plot(df_load.iloc[(week + lower_margin):(week + upper_margin)].values.flatten(), 'b')
plt.plot(grid_prof[24*(week + lower_margin):24*(week + upper_margin)], 'r')
# Load = Solar + Grid - (-Discharge + Charge)
# 2.4 = 5.5 + Grid - 3.0
#plt.plot(map(sum, zip(grid_prof[100*24:24*105],[x*-1 for x in action_list[24*100:24*105]], df_solar.iloc[100:105].values.flatten().tolist())), 'r')
plt.xlabel('Time in Hours')
plt.ylabel('Power kW')
plt.title('Weekday')
plt.show()

#### bins = np.arange(-3.,3.0, 0.5)
print bins
bins[np.digitize(2.3, bins, right=True)]

In [ ]:
np.arange(-2.6,5.201,0.40)